<a href="https://colab.research.google.com/github/JoaoADuarte/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install transformers datasets peft

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [3]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Configurar o ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

# Simular um navegador real com cabeçalhos personalizados
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")
driver = webdriver.Chrome(options=chrome_options)

conn = sqlite3.connect("chatbot_jardinagem_db")
cursor = conn.cursor()

# Nossa tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS artigos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fonte TEXT,
    titulo TEXT,
    conteudo TEXT,
    tags TEXT
);
""")
conn.commit()
print("DB criado!")


# Insere dados no BD
def inserir_dados(fonte, titulo, conteudo, tags):
  cursor.execute("""
  INSERT INTO artigos (fonte, titulo, conteudo, tags)
  VALUES (?,?,?,?)
  """, (fonte, titulo, conteudo, tags))
  conn.commit()

# Busca os dados do nosso DB
def buscar_conteudo(palavra_chave):
    cursor.execute("""
    SELECT titulo, conteudo FROM artigos
    WHERE tags LIKE ? OR conteudo LIKE ?
    """, (f"%{palavra_chave}%", f"%{palavra_chave}%"))
    resultados = cursor.fetchall()
    return resultados



# URLs das páginas para scraping
urls = [
    "https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html",
    "https://www.petz.com.br/blog/plantas/como-plantar-tomate/"
]

for url in urls:
    driver.get(url)
    time.sleep(3)  # Esperar a página carregar


    # Forçando atualização da página para evitar que reutilize dados
    driver.refresh()
    time.sleep(3)
    print(f"\n Extraindo dados de: {url}")

    # Título do artigo, onde pega o "h1" do HTML como título
    try:
      titulo_element = driver.find_element(By.TAG_NAME, "h1")
      titulo = titulo_element.text
    except:
      titulo = "Título não encontrado"

    # Extrair texto principal do site (ajustar conforme necessário)
    elementos_texto = driver.find_elements(By.TAG_NAME, "p")  # Pegar todos os parágrafos
    conteudo = "\n".join([e.text for e in elementos_texto if e.text.strip()])

    # Mostrar trecho do conteúdo extraído
    print(f"{titulo} - {conteudo[:300]}...\n")

    # Gerar tags com base no título de forma automática
    tags = ", ".join(titulo.lower().split())

    # Inserir dados no BD
    inserir_dados(url, titulo, conteudo, tags)
    print(f"Dados inseridos com sucesso em {url}")

# Fechar o navegador
driver.quit()

# Testando a busca
palavra_chave = "tomate"
resultados = buscar_conteudo(palavra_chave)

# Exibir os primeiros 500 caracteres de cada site
print("\nResultados da busca:\n")
for titulo, conteudo in resultados:
    print(f"{titulo}\n{conteudo[:500]}...\n")  # Mostra um trecho do conteúdo

# Fechar a conexão com o banco de dados
conn.close()

DB criado!

 Extraindo dados de: https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html
COMO PLANTAR TOMATE: OS SEGREDOS DO PLANTIO ATÉ A COLHEITA - Guia de Jardinagem
Atualizado em 21 de janeiro de 2024
Certamente todos sabemos que os tomates são um dos alimentos mais afetados por agrotóxicos. Segundo a ANVISA (Agência Nacional de Vigilância Sanitária) os tomates possuem resíduos acima do permitido. Mas existe uma saída, plantar tomate na sua c...

Dados inseridos com sucesso em https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html

 Extraindo dados de: https://www.petz.com.br/blog/plantas/como-plantar-tomate/
Descubra como plantar tomate com algumas dicas - Uma das principais hortaliças produzidas e consumidas no Brasil, o tomate é o “queridinho” de toda refeição. Afinal, ele vai bem com salada, pode se transformar em molho, sopa e muito mais. Porém, você já pensou em aprender como plantar tomate?
Além de um passa

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

modelo_nome = "google/flan-t5-large"

# Carregar o modelo e o tokenizador
tokenizer = T5Tokenizer.from_pretrained(modelo_nome)
modelo = T5ForConditionalGeneration.from_pretrained(modelo_nome)

 # Criação da função para pré-processamento das perguntas, alimentar o FLAN-T5 e gerar uma resposta.
def gerar_resposta(pergunta, contexto):
    prompt = f"""
    Você é um especialista em jardinagem.
    Responda a pergunta de forma clara e objetiva com base apenas no contexto abaixo:

    Contexto:
    {contexto}

    Pergunta: {pergunta}
    Resposta:
    """

    # Tokenização correta com o prompt completo
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Gerar a resposta com temperatura controlada para evitar repetições
    resposta_ids = modelo.generate(
        **inputs,
        max_length=200,  # Aumentar o comprimento máximo da resposta
        num_beams=8,     # Usar mais "beams" para melhorar a busca
        temperature=0.7, # Controlar a criatividade (valores menores = mais determinístico)
        top_p=0.9,       # Usar amostragem com nucleus sampling
        early_stopping=True,
        no_repeat_ngram_size=3  # Evitar repetições de 3+ palavras
    )
    # Decodificar a resposta
    resposta = tokenizer.decode(resposta_ids[0], skip_special_tokens=True)

    return resposta

def buscar_e_responder(pergunta):
    # Conectar ao banco de dados
    conn = sqlite3.connect("chatbot_jardinagem_db")
    cursor = conn.cursor()

    # Dividir a pergunta em palavras-chave
    palavras_chave = pergunta.lower().split()

    # Criar a consulta SQL mais eficiente
    query = """
    SELECT titulo, conteudo FROM artigos
    WHERE conteudo LIKE ?
    ORDER BY LENGTH(conteudo) ASC
    LIMIT 3
    """
    valores = [f"%{pergunta}%"]  # Buscar a pergunta completa primeiro

    cursor.execute(query, valores)
    resultados = cursor.fetchall()

    conn.close()

    if not resultados:
        return "Não encontrei informações suficientes sobre isso."

    # Selecionar os 500 primeiros caracteres do melhor resultado
    contexto = "\n".join([f"{titulo}: {conteudo[:500]}" for titulo, conteudo in resultados])

    # Gerar resposta com FLAN-T5
    resposta = gerar_resposta(pergunta, contexto)

    return resposta



pergunta = "Como plantar tomate?"
resposta = buscar_e_responder(pergunta)
print("🤖 Chatbot:", resposta)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


🤖 Chatbot: Para plantar tomate, siga os seguintes passos: 1. Escolha um local ensolado com pelo menos 6 horas de luz solar direta por dia. 2. Prepare o solo con adubo orgânico e certifique-se de que ele tenha boa drenagem. 3. Plante as mudas de tomate com espaçamento adequado (cerca de 50 cm entre plantas). 4. Regue regularmente, mantendo
